In [2]:
import pandas as pd
import pyensembl

In [6]:
def find_rows_with_gene_id(bed_file, gene_id):
    matching_rows = []

    with open(bed_file, 'r') as file:
        for line in file:
            columns = line.split('\t')
            if len(columns) >= 4 and columns[3] == gene_id:
                matching_rows.append(columns)

    if len(matching_rows) > 1:
        print("found more than one line with this gene id")
    if len(matching_rows) == 0:
        print("gene id not found in readthrough.bed")
    return matching_rows[0]

In [14]:
def xlsx_to_bed(xlsx_file, readthrough_or_readin_bed_file, outputfile):  
    data = pyensembl.Genome(reference_name='GRCh38', annotation_name='my_genome_features',
                                gtf_path_or_url='/data01/private/resources/gencode.v34.GRCh38.annotation.gtf')
    data.index()  # parse GTF and construct database of genomic features
    df = pd.read_excel(xlsx_file)
    #print(df)
    # Open the file for writing
    with open(outputfile, 'w') as bed_file:
        # Iterate over each row of the dataframe
        for index, row in df.iterrows():
            ens_id = df.iloc[index, 0]
            name = data.gene_name_of_gene_id(ens_id)
            bed_info = find_rows_with_gene_id(readthrough_or_readin_bed_file, ens_id)
            # Write the chromosome name, start position, end position, gene name,
            # score and strand information to the BED file
            bed_file.write('\t'.join([
                str(bed_info[0]),
                str(bed_info[1]),
                str(bed_info[2]),
                str(name),
                str(df.iloc[index, 6]), # FPKM
                str(bed_info[5])
            ]))


In [15]:
xlsx_to_bed("/home/ls/parshas/vscode/5465_processing/for_filter_by_fpkm/readthrough_filter_genecount_or_readthroughcount_or_geneFPKM_is0.xlsx","/private/projects/kidney_rtt/tcga_5465_output_one_sample_running_before_downsample/preprocess_files/readthrough.bed", "/home/ls/parshas/vscode/5465_processing/for_filter_by_fpkm/readthrough_filtered.bed")
xlsx_to_bed("/home/ls/parshas/vscode/5465_processing/for_filter_by_fpkm/read_in_filter_genecount_or_readincount_or_geneFPKM_is0.xlsx","/private/projects/kidney_rtt/tcga_5465_output_one_sample_running_before_downsample/preprocess_files/read_in.bed", "/home/ls/parshas/vscode/5465_processing/for_filter_by_fpkm/read_in_filtered.bed")